#  Gradio

https://gradio.app/

Gradio is the fastest way to demo your machine learning model with a friendly web interface so that anyone can use it, anywhere!


In [2]:
#!pip install gradio
#!pip install tensorflow==2.12.0
#!pip install keras==2.12.0
#!markupsafe==2.0.1

In [ ]:
import numpy as np
from keras.models import load_model

model = load_model('model.h5')

def classify_image(image):
    
    image = np.expand_dims(image, axis=0)
    prediction = model.predict(image)

    # Obtiene las etiquetas de las clases
    class_labels = ['plane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

    # Devuelve un diccionario con las etiquetas de las clases y sus probabilidades correspondientes
    return {class_labels[i]: float(prediction[0][i]) for i in range(10)}

In [ ]:
import gradio as gr

iface = gr.Interface(
    fn=classify_image,  # la función que hace la clasificación
    inputs=gr.inputs.Image(shape=(32, 32)),  # el tipo de entrada que espera tu modelo
    outputs=gr.outputs.Label(num_top_classes=3),  # el tipo de salida que produce tu modelo
)
iface.launch(share=True)

In [ ]:
#pip uninstall markupsafe

# Quantization

La cuantificación se refiere a reducir la precisión numérica de los pesos y las activaciones de un modelo, lo que permite reducir el tamaño y mejorar la eficiencia del modelo. 

TensorFlow proporciona herramientas y APIs para realizar la cuantificación de manera sencilla.

In [ ]:
#!pip install tensorflow_model_optimization

In [ ]:
import tensorflow as tf
from tensorflow_model_optimization.sparsity import keras as sparsity

converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
quantized_tflite_model = converter.convert()

with open('quantized_model.tflite', 'wb') as f:
    f.write(quantized_tflite_model)

#### Probamos: quantization

In [ ]:
def classify_image_2(image):
    
    image = np.expand_dims(image, axis=0)

    interpreter = tf.lite.Interpreter(model_path="quantized_model.tflite")
    interpreter.allocate_tensors()

    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()

    interpreter.set_tensor(input_details[0]['index'], image)

    # Ejecuta la inferencia
    interpreter.invoke()

    # Obtiene el resultado de la inferencia
    prediction = interpreter.get_tensor(output_details[0]['index'])

    # Obtiene las etiquetas de las clases
    class_labels = ['plane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

    # Devuelve un diccionario con las etiquetas de las clases y sus probabilidades correspondientes
    return {class_labels[i]: float(prediction[0][i]) for i in range(10)}

In [ ]:
import gradio as gr

iface = gr.Interface(
    fn=classify_image_2,  # la función que hace la clasificación
    inputs=gr.inputs.Image(shape=(32, 32)),  # el tipo de entrada que espera tu modelo
    outputs=gr.outputs.Label(num_top_classes=3),  # el tipo de salida que produce tu modelo
)
iface.launch(share=True)

# Pruning

El pruning, o poda, es una técnica utilizada para reducir el tamaño de un modelo eliminando los pesos que son pequeños o cero.

El concepto es similar a la poda en la jardinería, donde se eliminan las ramas innecesarias para mantener el árbol saludable.

Esto puede mejorar la eficiencia del modelo en términos de velocidad y tamaño de almacenamiento, a veces con un coste mínimo en términos de precisión.

In [ ]:
#!pip install tensorflow-model-optimization

In [4]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.models import Sequential
import numpy as np
import matplotlib.pyplot as plt
from tensorflow_model_optimization.sparsity import keras as sparsity

# Cargar y normalizar el conjunto de datos 
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Convertir las etiquetas en one-hot
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

# Define el preprocesamiento de la imagen

# Define tu modelo
model = tf.keras.models.Sequential([
 
    tf.keras.layers.Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(32, 32, 3)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(32, (3, 3), padding='same', activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Dropout(0.3),
    
    tf.keras.layers.Conv2D(64, (3, 3), padding='same', activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(64, (3, 3), padding='same', activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Dropout(0.5),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(10, activation='softmax')
])

# Compila y entrena el modelo
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

#history = model.fit(x_train, y_train, batch_size=256, epochs=1, validation_data=(x_test, y_test))

# Definir el esquema de pruning. En este caso, comenzamos sin pruning y terminamos con el 50% de los pesos pruned. 
# El pruning comienza en el paso 2000 y termina en el paso 4000.
pruning_params = {
      'pruning_schedule': sparsity.PolynomialDecay(initial_sparsity=0.0,
                                                   final_sparsity=0.5,
                                                   begin_step=2000,
                                                   end_step=4000)
}

# Envolver el modelo con las capas de pruning. Esto agregará una operación de pruning a cada capa de nuestro modelo.
model_for_pruning = sparsity.prune_low_magnitude(model, **pruning_params)

# Necesitamos recompilar el modelo después de agregar las capas de pruning.
model_for_pruning.compile(optimizer='adam', 
                          loss='categorical_crossentropy',
                          metrics=['accuracy'])

# Entrenar el modelo con callbacks para habilitar pruning. 
# El callback UpdatePruningStep asegurará que el estado del pruning se actualiza en cada paso de entrenamiento.
callbacks = [
  sparsity.UpdatePruningStep(),
]

# Ajustar el modelo con nuestros datos de entrenamiento. Durante este entrenamiento, los pesos del modelo serán pruned según el esquema que definimos antes.
model_for_pruning.fit(x_train, y_train,
                      epochs=1,
                      callbacks=callbacks)

# Guarda el modelo#
model_for_pruning.save('pruning_conv.h5')

2023-10-19 23:49:47.210093: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 153600000 exceeds 10% of free system memory.


1563/1563 [==============================] - 336s 210ms/step - loss: 1.5877 - accuracy: 0.4401
